# Tokenize the data (Contexts, questions, answer start and end index)

In [18]:
!pip install transformers
import torch
import json
import json
from pathlib import Path
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import pickle

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
training_contexts, training_questions, training_ans = [], [], []
validation_contexts, validation_questions, validation_ans = [], [], []

with open('SQUAD2.0_data/modified/train_contexts-modified.json', 'rb') as f:
    training_contexts = json.load(f)

with open('SQUAD2.0_data/modified/train_ques-modified.json', 'rb') as f:
    training_questions = json.load(f)

with open('SQUAD2.0_data/modified/train_ans-modified.json', 'rb') as f:
    training_ans = json.load(f)

with open('SQUAD2.0_data/modified/dev_contexts-modified.json', 'rb') as f:
    validation_contexts = json.load(f)

with open('SQUAD2.0_data/modified/dev_ques-modified.json', 'rb') as f:
    validation_questions = json.load(f)

with open('SQUAD2.0_data/modified/dev_ans-modified.json', 'rb') as f:
    validation_ans = json.load(f)


## BERT base uncased

In [8]:
### We will be using uncased model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

## Tokenize validation contexts

In [9]:
training_batch = tokenizer(training_contexts, training_questions, truncation=True, padding=True)

## Tokenize validation contexts

In [10]:
validation_batch = tokenizer(validation_contexts, validation_questions, truncation=True, padding=True)

In [12]:
answer_start_positions, answer_end_positions = [], []

In [13]:
default = tokenizer.model_max_length
def update_start_end_tokens(batch, ans):
  for i in range(len(ans)):
    if batch.char_to_token(i, ans[i]['answer_start']) is not None:
      answer_start_positions.append(batch.char_to_token(i, ans[i]['answer_start']))
    else:
      answer_start_positions.append(default)
    if batch.char_to_token(i, ans[i]['answer_end']) is not None:
      answer_end_positions.append(batch.char_to_token(i, ans[i]['answer_end']))
    elif batch.char_to_token(i, ans[i]['answer_end']-1) is not None:
      answer_end_positions.append(batch.char_to_token(i, ans[i]['answer_end']-1))
    else:
      answer_end_positions.append(default)
  batch.update({'start_positions': answer_start_positions, 'end_positions': answer_end_positions})


In [14]:
answer_start_positions, answer_end_positions = [], [] 
update_start_end_tokens(training_batch, training_ans)
answer_start_positions, answer_end_positions = [], [] 
update_start_end_tokens(validation_batch, validation_ans)

In [21]:
with open('SQUAD2.0_data/tokenized/training_tokenized.pickle', 'wb') as output:
    pickle.dump(training_batch, output)

with open('SQUAD2.0_data/tokenized/validation_tokenized.pickle', 'wb') as output:
    pickle.dump(validation_batch, output)